In [ ]:
#import katdal
import requests
from matplotlib import pyplot as plt
import numpy as np
import calendar
import matplotlib.dates as mdates
import datetime
from scipy.interpolate import interp1d
import pandas
import tempfile
import ipywidgets as widgets
from IPython.display import display
from ast import literal_eval as make_tuple
from scipy.signal import convolve

def getdata(sensor_name='m000_dig_l_band_rfcu_hpol_rf_power_in',
            sensors_url = 'http://kat-flap-cpt.mkat.control.kat.ac.za//katstore/api/query',
            start = [calendar.timegm((2019,12,18,22,0,0))],
            end = [calendar.timegm((2019,12,18,23,0,0))],
            interval=None,
            ack=True
           ):
    assert len(start) == len(end)
    samples = []
    sample_times = []
    if ack:
        progress = widgets.IntProgress(value=0, max=len(start))
        labelprogress = widgets.HBox([widgets.Label("Getting " + sensor_name), 
                                      progress])
        display(labelprogress)
    for tei, (ss, se) in enumerate(zip(start, end)):
        sample_params = {'sensor':sensor_name,
                         'start_time': ss,
                         'end_time': se,
                         'include_value_time': True,
                        }
        if interval is not None:
            sample_params['interval'] = interval
        try:
            resp = requests.get(sensors_url, sample_params)
        except Exception as exc:
            print('Something failed: {}'.format(exc))
            raise exc

        if resp.status_code == 200:
            sample_results = resp.json()
            if interval is not None:
                samples += [x['avg_value'] for x in sample_results['data']]
                this_times = [x['min_value_time'] for x in sample_results['data']]
                sample_times += this_times
            else:
                samples += [x['value'] for x in sample_results['data']]
                this_times = [x['sample_time'] for x in sample_results['data']]
                sample_times += this_times
        else:
            raise RuntimeError("Request returned with a status code {}".format(resp.status_code))
        if ack:
            progress.value += 1
    return np.array(list(zip(sample_times, samples))).reshape(len(sample_times), 2)

In [ ]:
# gain calibrator scans
# must be a csv in the format e.g.
# start,end,tracking,scanlabel,dumps,target\n
# 2021/03/19 03:00:57,2021/03/19 03:02:41,3:track,1:track,14,1:J2011-0644\n
# ...
with tempfile.NamedTemporaryFile(mode="w+") as t:
    t.write(
            """
              start,end,tracking,scanlabel,dumps,target\n
              2021/03/19 03:00:57,2021/03/19 03:02:41,3:track,1:track,14,1:J2011-0644\n
              2021/03/19 03:43:44,2021/03/19 03:45:28,7:track,3:track,14,1:J2011-0644\n
              2021/03/19 04:26:31,2021/03/19 04:28:15,11:track,5:track,14,1:J2011-0644\n
              2021/03/19 05:09:26,2021/03/19 05:11:10,15:track,7:track,14,1:J2011-0644\n
              2021/03/19 05:52:13,2021/03/19 05:54:05,19:track,9:track,15,1:J2011-0644\n
              2021/03/19 06:35:24,2021/03/19 06:37:08,23:track,11:track,14,1:J2011-0644\n
            """
           )
    t.seek(0)
    t.flush()
    scans = pandas.read_csv(t.name)
scans

In [ ]:
antsels=["m037","m000","m010"]

# Plots
### Pointing

In [ ]:
dt_to_tgm = lambda tt: calendar.timegm((
    tt.timetuple().tm_year, tt.timetuple().tm_mon, tt.timetuple().tm_mday,
    tt.timetuple().tm_hour, tt.timetuple().tm_min, tt.timetuple().tm_sec))
start_key = list(filter(lambda x: x.strip() == 'start', scans.keys()))[0]
end_key = list(filter(lambda x: x.strip() == 'end', scans.keys()))[0]
scan_times = [(dt_to_tgm(datetime.datetime.strptime(ss.strip(), '%Y/%m/%d %H:%M:%S')),
               dt_to_tgm(datetime.datetime.strptime(se.strip(), '%Y/%m/%d %H:%M:%S'))
              ) for ss, se in zip(scans[start_key], scans[end_key])]

In [ ]:
npltx = int(np.ceil(np.sqrt(len(antsels))))
nplty = len(antsels) // npltx + (len(antsels) % npltx >= 1)
fig, axs = plt.subplots(nplty, npltx, subplot_kw={'projection': 'polar'}, figsize=(7 * nplty, 7 * npltx))
for antsel, ax in zip(antsels, axs.ravel()):
    azact = getdata("{}_pos_actual_scan_azim".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)), interval=1)
    elevact = getdata("{}_pos_actual_scan_elev".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)), interval=1)
    assert len(azact) == len(elevact)
    xtime = azact[:, 0]
    azactLERP = interp1d(azact[:, 0], azact[:, 1], fill_value="extrapolate")
    elevactLERP = interp1d(elevact[:, 0], elevact[:, 1], fill_value="extrapolate")
    ax.scatter(np.deg2rad(azactLERP(xtime)), elevactLERP(xtime))
    ax.set_rmax(90.0)
    ax.set_title(antsel)
    ax.grid(True)
for ax in axs.ravel()[len(antsels):]:
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
hfmt = mdates.DateFormatter('%H:%M')
for antsel in antsels:
    azact = getdata("{}_pos_actual_scan_azim".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)), interval=1)
    elevact = getdata("{}_pos_actual_scan_elev".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)), interval=1)
    azreq = getdata("{}_pos_request_scan_azim".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)), interval=1)
    elevreq = getdata("{}_pos_request_scan_elev".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)), interval=1)
    assert len(azact) == len(elevact)
    assert len(azreq) == len(elevreq)
    if azact.size == 0 or azreq.size == 0:
        continue
    xtime = azact[:, 0]
    azactLERP = interp1d(azact[:, 0], azact[:, 1], fill_value="extrapolate")
    elevactLERP = interp1d(elevact[:, 0], elevact[:, 1], fill_value="extrapolate")
    azreqLERP = interp1d(azreq[:, 0], azreq[:, 1], fill_value="extrapolate")
    elevreqLERP = interp1d(elevreq[:, 0], elevreq[:, 1], fill_value="extrapolate")
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), xtime)),
             (azactLERP(xtime) - azreqLERP(xtime))*3600,
             marker="x",
             label=antsel+"_az") 
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), xtime)),
             (elevactLERP(xtime) - elevreqLERP(xtime))*3600,
             marker="x",
             label=antsel+"_elev") 
plt.xlabel("time [utc] start {}".format(np.min(list(map(lambda x: datetime.datetime.utcfromtimestamp(x[0]), scan_times)))))
plt.ylabel("Pointing offset from requested [asec]")
plt.title("Pointing offsets from requested")
plt.gca().xaxis.set_major_formatter(hfmt)
plt.legend()
plt.grid(True)
plt.show()

### Power levels ADC and digtitizer

In [ ]:
plt.figure(figsize=(12, 6))
hfmt = mdates.DateFormatter('%H:%M')
for antsel in antsels:
    hpol = getdata("{}_dig_l_band_rfcu_hpol_rf_power_in".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    vpol = getdata("{}_dig_l_band_rfcu_vpol_rf_power_in".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    if hpol.size == 0 or vpol.size == 0:
        continue
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), hpol[:, 0])),
             hpol[:, 1],
             marker="x",
             label=antsel+"_hpol") 
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), vpol[:, 0])),
             vpol[:, 1],
             marker="x",
             label=antsel+"_vpol") 
plt.xlabel("time [utc] start {}".format(np.min(list(map(lambda x: datetime.datetime.utcfromtimestamp(x[0]), scan_times)))))
plt.ylabel("Power [dB]")
plt.title("Rx power out")
plt.gca().xaxis.set_major_formatter(hfmt)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
hfmt = mdates.DateFormatter('%H:%M')
for antsel in antsels:
    hpol = getdata("{}_dig_l_band_adc_hpol_rf_power_in".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    vpol = getdata("{}_dig_l_band_adc_hpol_rf_power_in".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    if hpol.size == 0 or vpol.size == 0:
        continue
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), hpol[:, 0])),
             hpol[:, 1],
             marker="x",
             label=antsel+"_hpol") 
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), vpol[:, 0])),
             vpol[:, 1],
             marker="x",
             label=antsel+"_vpol") 
plt.xlabel("time [utc] start {}".format(np.min(list(map(lambda x: datetime.datetime.utcfromtimestamp(x[0]), scan_times)))))
plt.ylabel("Power [dB]")
plt.title("ADC power")
plt.gca().xaxis.set_major_formatter(hfmt)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
hfmt = mdates.DateFormatter('%H:%M')
for antsel in antsels:
    rx_temp = getdata("{}_rsc_rxl_rfe1_temperature".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    if rx_temp.size == 0:
        continue
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), rx_temp[:, 0])),
             rx_temp[:, 1],
             marker="x",
             label=antsel) 
    
plt.xlabel("time [utc] start {}".format(np.min(list(map(lambda x: datetime.datetime.utcfromtimestamp(x[0]), scan_times)))))
plt.ylabel("Temp [K]")
plt.title("RxL temperature")
plt.gca().xaxis.set_major_formatter(hfmt)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
hfmt = mdates.DateFormatter('%H:%M')
for antsel in antsels:
    lnahdc = getdata("{}_rsc_rxl_lna_h_drain_current".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    lnavdc = getdata("{}_rsc_rxl_lna_v_drain_current".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    if lnahdc.size == 0 or lnavdc.size == 0:
        continue
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), lnahdc[:, 0])),
             lnahdc[:, 1],
             marker="x",
             label=antsel+" vpol") 
    plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), lnavdc[:, 0])),
             lnavdc[:, 1],
             marker="x",
             label=antsel+" vpol") 
    
plt.xlabel("time [utc] start {}".format(np.min(list(map(lambda x: datetime.datetime.utcfromtimestamp(x[0]), scan_times)))))
plt.ylabel("LNA Drain current [mA]")
plt.title("LNA Drain currents")
plt.gca().xaxis.set_major_formatter(hfmt)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
hfmt = mdates.DateFormatter('%H:%M')
for antsel in antsels:
    lnahdc = getdata("{}_rsc_rxl_lna_h_drain_voltage".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    lnavdc = getdata("{}_rsc_rxl_lna_v_drain_voltage".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    if lnahdc.size == 0 or lnavdc.size == 0:
        continue
    plt.scatter(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), lnahdc[:, 0])),
             lnahdc[:, 1],
             marker="x",
             label=antsel+" vpol") 
    plt.scatter(list(map(lambda x: datetime.datetime.utcfromtimestamp(x), lnavdc[:, 0])),
             lnavdc[:, 1],
             marker="x",
             label=antsel+" vpol") 
    
plt.xlabel("time [utc] start {}".format(np.min(list(map(lambda x: datetime.datetime.utcfromtimestamp(x[0]), scan_times)))))
plt.ylabel("LNA Drain current [mV]")
plt.title("LNA Drain currents")
plt.gca().xaxis.set_major_formatter(hfmt)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
hfmt = mdates.DateFormatter('%H:%M')

for antsel in antsels:
    hpol = getdata("{}_dig_l_band_rfcu_hpol_rf_power_in".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    vpol = getdata("{}_dig_l_band_rfcu_vpol_rf_power_in".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)))
    elev = getdata("{}_pos_actual_scan_elev".format(antsel), start=list(map(lambda x: x[0], scan_times)), end=list(map(lambda x: x[1], scan_times)), interval=1)
    if hpol.size == 0 or vpol.size == 0 or elev.size == 0:
        continue
    elev = interp1d(elev[:, 0], elev[:, 1], fill_value="extrapolate")
    
    plt.plot(elev(hpol[:, 0]),
             hpol[:, 1],
             marker="x",
             label=antsel+"_hpol") 
    
    plt.plot(elev(vpol[:, 0]),
             vpol[:, 1],
             marker="x",
             label=antsel+"_vpol") 
    
plt.xlabel("AP actual elevation [deg]")
plt.ylabel("Power [dB]")
plt.title("Rx power")
plt.legend()
plt.grid(True)
plt.show()

### Delays

In [ ]:
plt.figure(figsize=(12, 6))
hfmt = mdates.DateFormatter('%H:%M')
for cbf in np.arange(4) + 1:
    for antsel in antsels:
        # what is the correlator applying?
        # JM says index is
        # 0 - adc sample
        # 1 - delay
        # 2 - rate
        # 3 - phase offset
        # 4 - phase offset rate
        delay = getdata("cbf_{}_wide_antenna_channelised_voltage_{}v_delay".format(cbf, antsel), 
                        start=list(map(lambda x: x[0], scan_times)), 
                        end=list(map(lambda x: x[1], scan_times)))
        if delay.size == 0:
            continue
        plt.plot(list(map(lambda x: datetime.datetime.utcfromtimestamp(int(np.round(float(x)))), delay[:, 0])),
                 list(map(lambda x: make_tuple(x)[1] * 1e6, delay[:, 1])),
                 marker="x",
                 label="cbf{}.{}".format(cbf, antsel)) 
    
plt.xlabel("time [utc] start {}".format(np.min(list(map(lambda x: datetime.datetime.utcfromtimestamp(x[0]), scan_times)))))
plt.ylabel("Delay applied [ms]")
plt.gca().xaxis.set_major_formatter(hfmt)
plt.legend()
plt.grid(True)
plt.show()